### Install Dependencies

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

### Download Correctly Formatted Custom Dataset

In [ ]:
# Export code snippet and paste here
%cd /home/zjd/Yolo
#!curl -L "https://public.roboflow.com/ds/8ACXCZKUsQ?key=xUz6VxCnMm" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!unzip Synthetic\ Fruit.v1-raw.yolov5pytorch.zip

In [1]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['mask', 'no_mask']

### Define Model Configuration and Architecture

In [2]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [3]:
#this is the model configuration we will use for our tutorial 
%cat ./yolov5/models/yolov5l.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1,

In [4]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [5]:
%%writetemplate ./yolov5/models/custom_yolov5l.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### Train Custom YOLOv5 Detector

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [6]:
# train yolov5s on custom data for 100 epochs
# time its performance
%time
%cd ./yolov5/
!python train.py --img 416 --batch 16 --epochs 300 --data '../data.yaml' --cfg ./models/custom_yolov5l.yaml --weights './weights/yolov5l.pt' --name yolov5l_results  --cache

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs
/home/zjd/Yolo/yolov5
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 164 (delta 95), reused 89 (delta 89), pack-reused 59
接收对象中: 100% (164/164), 344.71 KiB | 79.00 KiB/s, 完成.
处理 delta 中: 100% (113/113), 完成 29 个本地对象.
来自 https://github.com/ultralytics/yolov5
   d11504a..bd5cfff  master     -> origin/master
 * [新标签]          v3.1       -> v3.1
Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1080', total_memory=8111MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/custom_yolov5l.yaml', data='../data.yaml', device='', epochs=300, evolve=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], local_rank=-1, logdir='runs/', multi_scale=False, name='yolov5l_results', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, singl


     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    32/299     4.57G   0.02859   0.05001  0.001699    0.0803        33       416
               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.643       0.872       0.842       0.451

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    33/299     4.57G   0.02855    0.0491  0.001699   0.07936         7       416
               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.607       0.874       0.825        0.46

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    34/299     4.57G   0.02699   0.04825  0.001501   0.07673         7       416
               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.621       0.862       0.824  

               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.732       0.841       0.822       0.439

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    81/299     4.57G   0.02135   0.03917 0.0008486   0.06136        13       416
               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.716       0.829       0.805       0.439

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    82/299     4.57G   0.02115   0.03855 0.0008383   0.06054         4       416
               Class      Images     Targets           P           R      mAP@.5
                 all          95         481       0.708       0.861        0.83       0.457

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
    83/299     4.57G   0.02116   0.04019 0.0007792   0.06213         9

### Evaluate Custom YOLOv5 Detector Performance

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.general import plot_results;
#plot_results()  # plot results.txt as results.png
Image(filename='./runs/exp0_yolov5l_results/results.png', width=1000)  # view results.png

#### Curious? Visualize Our Training Data with Labels

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='./runs/exp0_yolov5l_results/test_batch0_gt.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='./runs/exp0_yolov5l_results/train_batch2.jpg', width=900)

### Run Inference With Trained Weights Run inference with a pretrained checkpoint on contents of test/images folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/exp1_yolov5l_results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5/
!python detect.py --weights runs/exp0_yolov5l_results/weights/best.pt --img 416 --conf 0.1 --source ../test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/inference/output/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

### Export Trained Weights for Future Inference
Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp /content/yolov5/weights/last_yolov5s_results.pt /content/gdrive/My\ Drive